In [1]:
import sys
assert sys.version_info >= (3, 5)

In [2]:
import numpy as np
import os

In [3]:
HOUSING_PATH = os.path.join("datasets")

In [4]:
import pandas as pd

names = ["Class", "Alcohol", "Malic acid", "Ash", "Alcalinity of ash", "Magnesium", "Total phenols", "Flavanoids",
         "Nonflavanoid phenols", "Proanthocyanins", "Color intensity", "Hue", "OD280/OD315 of diluted wines", "Proline"]

def load_housing_data(housing_path=HOUSING_PATH):
    path = os.path.join(housing_path, "wine.data")
    return pd.read_csv(path, header=None, names=names)


In [5]:
wine = load_housing_data()
wine.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [6]:
wine["Class"].value_counts()

2    71
1    59
3    48
Name: Class, dtype: int64

### 2 Извлеките из данных признаки и классы.

Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)

In [7]:
X = wine.loc[:, wine.columns != "Class"]
y = wine["Class"]


### 3. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold).

Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

### 4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50.

При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

def get_best_score(X: pd.DataFrame, y: pd.Series, cv):
    best_score, best_k = None, None

    for k in range(1, 51):
        model = KNeighborsClassifier(n_neighbors=k)
        score = cross_val_score(model, X, y, cv=cv, scoring="accuracy").mean()
        
        if best_score is None or score > best_score:
            best_score, best_k = score, k
    
    return best_score, best_k

### 5. Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale.

Снова найдите оптимальное k на кросс-валидации.

In [10]:
score, k = get_best_score(X, y, kf)
print(k)
print(score)

1
0.7304761904761905


### 6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу?

Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

In [11]:
from sklearn.preprocessing import scale

score, k = get_best_score(scale(X), y, kf)

print(k)
print(score)

29
0.9776190476190475
